# Analyse complète : Séismes & Prix Immobiliers aux États-Unis  
Ce notebook regroupe :
- Deux cartes US (séismes & prix immobiliers)
- Un scatter avec régression
- Une analyse High/Low n_earthquake groupée
- Une visualisation centrée (taille de points)

In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")
import numpy as np
import us
from vega_datasets import data
import os

# Load dataset
df_state_aggreg = pd.read_csv("data/agg_state_year.csv")
df_county_aggreg = pd.read_csv("data/agg_county_year.csv")

# print(df_state_aggreg.head(), df_county_aggreg.head())

### Chargement données pour la carte

In [2]:
# ---------------------------
# LOAD US STATES TOPOJSON
# ---------------------------
us_states = alt.topo_feature(data.us_10m.url, "states")

## Sélection de l’année

In [3]:
year = 2015
df_state_aggreg_year = df_state_aggreg[df_state_aggreg["year"] == year]
df_county_aggreg_year = df_county_aggreg[df_county_aggreg["year"] == year]

# print(df_state_aggreg_year.head(), df_county_aggreg_year.head())

# Carte US : Intensité des Séismes et Prix Médian des Maisons

In [4]:
heatmap_eq = alt.Chart(us_states).mark_geoshape().encode(
    color=alt.Color(
        "n_earthquakes:Q",
        scale=alt.Scale(range=["#ffe6e6", "#800000"]),
        title="Number of earthquake"
    ),
    tooltip=["state:N", "n_earthquakes:Q"]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(df_state_aggreg, "fips", ["n_earthquakes", "state"])
).project("albersUsa").properties(
    title=f"USA earthquake map ({year})", width=400, height=300
)

heatmap_price = alt.Chart(us_states).mark_geoshape().encode(
    color=alt.Color(
        "avg_price:Q",
        scale=alt.Scale(range=["#e6f2ff", "#0055aa"]),
        title="Average real estate price"
    ),
    tooltip=["state:N", "avg_price:Q"]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(df_state_aggreg.dropna(subset=["avg_price"]), "fips", ["avg_price", "state"])
).project("albersUsa").properties(
    title="USA Average real estate price", width=400, height=300
)

(heatmap_eq | heatmap_price).resolve_scale(color="independent")

alt.HConcatChart(...)

In [5]:
heatmap_eq = alt.Chart(us_states).mark_geoshape().encode(
    color=alt.Color(
        "n_earthquakes:Q",
        scale=alt.Scale(range=["#ffe6e6", "#800000"]),
        title="Number of earthquake"
    ),
    tooltip=["State:N", "n_earthquakes:Q"]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(df_state_aggreg_year, "fips", ["n_earthquakes", "state"])
).project("albersUsa").properties(
    title=f"USA earthquake map ({year})", width=400, height=300
)

heatmap_price = alt.Chart(us_states).mark_geoshape().encode(
    color=alt.Color(
        "avg_price:Q",
        scale=alt.Scale(range=["#e6f2ff", "#0055aa"]),
        title="Average real estate price"
    ),
    tooltip=["state:N", "avg_price:Q"]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(df_state_aggreg_year, "fips", ["avg_price", "state"])
).project("albersUsa").properties(
    title=f"USA Average real estate price ({year})", width=400, height=300
)

(heatmap_eq | heatmap_price).resolve_scale(color="independent")

alt.HConcatChart(...)

# Corrélation Séismes ↔ Prix

## 🔎 Analyse de la Relation entre l’Activité Sismique et les Prix Immobiliers  
### Une visualisation combinant échelle logarithmique, intensité sismique et tendance générale

Cette visualisation explore comment le nombre de séismes dans un État américain est associé au prix médian de l’immobilier.  
Elle combine plusieurs éléments complémentaires :

- **Scatter plot** : chaque point représente un État pour l’année sélectionnée.  
- **Échelle logarithmique sur l’axe des X** : permet de visualiser correctement des niveaux de sismicité très différents (de 1 à plusieurs centaines).  
- **Couleur continue (Viridis)** : encode l’intensité des séismes, renforçant la lecture des variations.  
- **Regression Linéaire** : une tendance linéaire qui met en évidence la relation générale.  

L’objectif final est d’offrir une lecture claire, équilibrée et robuste de la relation potentielle entre l’activité sismique et les prix domiciliaires, malgré la grande variabilité des États.

In [6]:
# Filter out invalid values
df_corr = df_state_aggreg_year[
    (df_state_aggreg_year["n_earthquakes"] > 0) &
    (df_state_aggreg_year["avg_price"] > 0)
].copy()

# Ensure numeric
df_corr["n_earthquakes"] = pd.to_numeric(df_corr["n_earthquakes"], errors="coerce")
df_corr["avg_price"] = pd.to_numeric(df_corr["avg_price"], errors="coerce")

# Color scale
color_scale = alt.Scale(type="log", scheme="viridis")

scatter = (
    alt.Chart(df_corr)
    .mark_circle(size=90, opacity=0.75)
    .encode(
        x=alt.X("n_earthquakes:Q", title="Earthquakes (log scale)", scale=alt.Scale(type="log")),
        y=alt.Y("avg_price:Q", title="Average House Price ($)", scale=alt.Scale(zero=False)),
        color=alt.Color("n_earthquakes:Q", title="Earthquake Count", scale=color_scale,
                        legend=alt.Legend(labelExpr="datum.label")),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("n_earthquakes:Q", title="Earthquakes"),
            alt.Tooltip("avg_price:Q", title="Avg Price ($)")
        ]
    )
)

smooth = (
    alt.Chart(df_corr)
    .transform_regression("n_earthquakes", "avg_price", method="linear")
    .mark_line(color="black", size=3)
    .encode(
        x=alt.X("n_earthquakes:Q", scale=alt.Scale(type="log")),
        y="avg_price:Q"
    )
)

correlation = (scatter + smooth).properties(
    width=900,
    height=550,
    title="Earthquake Frequency vs House Prices (Log Scale)"
)

correlation

alt.LayerChart(...)

In [7]:
import altair as alt
import pandas as pd
import numpy as np

# --------------------------
# 1. FILTER AND CLEAN DATA
# --------------------------

df_corr = df_state_aggreg_year[
    (df_state_aggreg_year["n_earthquakes"] > 0) &
    (df_state_aggreg_year["avg_price"] > 0) &
    (df_state_aggreg_year["avg_magnitude"].notna())
].copy()

# Ensure numeric
df_corr["n_earthquakes"] = pd.to_numeric(df_corr["n_earthquakes"], errors="coerce")
df_corr["avg_price"] = pd.to_numeric(df_corr["avg_price"], errors="coerce")
df_corr["avg_magnitude"] = pd.to_numeric(df_corr["avg_magnitude"], errors="coerce")

# --------------------------
# 2. CREATE DYNAMIC EXCLUSIVE BINS
# --------------------------

# Determine min and max for avg_magnitude
min_mag = df_corr["avg_magnitude"].min()
max_mag = df_corr["avg_magnitude"].max()

# Create 5 equal-width bins between min and max
bins = np.linspace(min_mag, max_mag, num=6)  # 6 edges → 5 bins
labels = [f"{round(bins[i],1)}-{round(bins[i+1],1)}" for i in range(len(bins)-1)]

# right=False ensures exclusive upper boundary
df_corr["magnitude_bin"] = pd.cut(df_corr["avg_magnitude"], bins=bins, labels=labels, right=False)

# --------------------------
# 3. DEFINE DARK COLOR SCALE
# --------------------------

color_scale = alt.Scale(
    domain=labels,
    range=["#7f3c8d", "#11a579", "#3969ac", "#f2b701", "#e73f74"]  # dark, distinct
)

# --------------------------
# 4. SCATTER PLOT
# --------------------------

scatter = (
    alt.Chart(df_corr)
    .mark_circle(size=90, opacity=0.75)
    .encode(
        x=alt.X("n_earthquakes:Q", title="Earthquakes (log scale)", scale=alt.Scale(type="log")),
        y=alt.Y("avg_price:Q", title="Average House Price ($)", scale=alt.Scale(zero=False)),
        color=alt.Color("magnitude_bin:N", title="Avg Magnitude Bin", scale=color_scale),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("n_earthquakes:Q", title="Earthquakes"),
            alt.Tooltip("avg_price:Q", title="Avg Price ($)"),
            alt.Tooltip("avg_magnitude:Q", title="Avg Magnitude"),
            alt.Tooltip("magnitude_bin:N", title="Magnitude Bin")
        ]
    )
)

# --------------------------
# 5. REGRESSION TRENDLINE
# --------------------------

smooth = (
    alt.Chart(df_corr)
    .transform_regression("n_earthquakes", "avg_price", method="linear")
    .mark_line(color="black", size=3)
    .encode(
        x=alt.X("n_earthquakes:Q", scale=alt.Scale(type="log")),
        y="avg_price:Q"
    )
)

# --------------------------
# 6. COMBINE PLOT
# --------------------------

correlation = (scatter + smooth).properties(
    width=900,
    height=550,
    title="Earthquake Frequency vs House Prices (Log Scale, Dynamic Magnitude Bins)"
)

correlation


alt.LayerChart(...)

# Classification des États : High vs Low n_earthquake
Seuls les États dans les quantiles 20% et 80% sont conservés.

In [8]:
# Sum of earthquakes per state
state_totals = df_state_aggreg_year.groupby('state')['n_earthquakes'].sum().sort_values()

threshold_high = state_totals.quantile(0.8)
threshold_low = state_totals.quantile(0.2)

def classify_state(state):
    total = state_totals[state]
    if total >= threshold_high:
        return 'High n_earthquake'
    elif total <= threshold_low:
        return 'Low n_earthquake'
    else:
        return 'Medium'

# Apply classification
df_state_aggreg_year['quake_group'] = df_state_aggreg_year['state'].apply(classify_state)

# Select only high and low groups
df_selected = df_state_aggreg_year[df_state_aggreg_year['quake_group'].isin(['High n_earthquake', 'Low n_earthquake'])]

# df_selected.head()

/var/folders/lq/3djnzdbx6ll0000lzlm2th6w0000gn/T/ipykernel_7696/3623333408.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state_aggreg_year['quake_group'] = df_state_aggreg_year['state'].apply(classify_state)


# Evolution des prix selon High / Low n_earthquake

In [9]:
import altair as alt
import pandas as pd

# --- Filter data after 1990 ---
df_state_aggreg_filtered = df_state_aggreg[df_state_aggreg['year'] > 1990].copy()

# Y-axis domain
buffer = 1_000_000
ymin = df_state_aggreg_filtered['avg_price'].min()
ymax = df_state_aggreg_filtered['avg_price'].max() - buffer

# --- Clip scatter points within domain ---
df_state_aggreg_filtered = df_state_aggreg_filtered[
    (df_state_aggreg_filtered['avg_price'] >= ymin) &
    (df_state_aggreg_filtered['avg_price'] <= ymax)
].copy()

# --- Separate Alaska & California ---
df_alaska = df_state_aggreg_filtered[df_state_aggreg_filtered['state'] == 'Alaska']
df_california = df_state_aggreg_filtered[df_state_aggreg_filtered['state'] == 'California']

# --- Scatter with variable size ---
scatter = (
    alt.Chart(df_state_aggreg_filtered)
    .mark_circle(opacity=0.7)
    .encode(
        x=alt.X('year:O', title='Year'),
        y=alt.Y('avg_price:Q', title='Average House Price ($)',
                scale=alt.Scale(domain=[ymin, ymax])),
        size=alt.Size('n_earthquakes:Q', title='Number of Earthquakes',
                      scale=alt.Scale(range=[30, 400])),
        color=alt.value("#999999"),  # scatter = neutral grey
        tooltip=['state:N', 'year:O', 'n_earthquakes:Q', 'avg_price:Q']
    )
)

# --- Lines with legend ---
line_data = pd.concat([df_alaska, df_california])
line_data['line_label'] = line_data['state']  # used for the legend

line = (
    alt.Chart(line_data)
    .mark_line(size=3)
    .encode(
        x='year:O',
        y='avg_price:Q',
        color=alt.Color(
            'line_label:N',
            title='States',
            scale=alt.Scale(
                domain=['Alaska', 'California'],
                range=['#1f77b4', '#d62728']  # blue, red
            )
        ),
        tooltip=['state:N', 'year:O', 'avg_price:Q']
    )
)

# --- Combine ---
chart = (
    (scatter + line)
    .properties(
        width=900,
        height=500,
        title='House Prices Over Time – Alaska vs California (After 1990)'
    )
    .configure_axis(
        labelFontSize=12,
        titleFontSize=14,
        gridOpacity=0.2
    )
    .configure_legend(
        titleFontSize=14,
        labelFontSize=13,
        symbolSize=150
    )
    .configure_title(
        fontSize=18,
        anchor='start'
    )
)

chart

alt.LayerChart(...)

In [10]:
import altair as alt
import numpy as np

# --------------------------
# 1. CLEAN DATA
# --------------------------

df_corr = df_state_aggreg.copy()

# Remove invalid values
df_corr = df_corr[
    (df_corr["n_earthquakes"] > 0) &
    (df_corr["avg_price"] > 0)
].copy()

# Add log value for regression
df_corr["log_eq"] = np.log(df_corr["n_earthquakes"])

# --------------------------
# 2. SCATTER (log scale)
# --------------------------

color_scale = alt.Scale(scheme="viridis")

scatter_all = (
    alt.Chart(df_corr)
    .mark_circle(size=90, opacity=0.6)
    .encode(
        x=alt.X(
            "n_earthquakes:Q",
            title="Earthquakes (log scale)",
            scale=alt.Scale(type="log"),
            axis=alt.Axis(format="~s")
        ),
        y=alt.Y(
            "avg_price:Q",
            title="Median House Price ($)",
            scale=alt.Scale(zero=False, padding=10)
        ),
        color=alt.Color(
            "log_eq:Q",
            title="Earthquakes (log)",
            scale=color_scale
        ),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("n_earthquakes:Q", title="Earthquakes"),
            alt.Tooltip("avg_price:Q", title="House Price ($)")
        ]
    )
)

# --------------------------
# 3. TRENDLINE with log transform
# --------------------------

smooth_all = (
    alt.Chart(df_corr)
    .transform_regression(
        "log_eq", "avg_price"
    )
    .mark_line(
        color="black",
        size=3
    )
    .encode(
        x=alt.X(
            "n_earthquakes:Q",
            scale=alt.Scale(type="log")
        ),
        y="avg_price:Q"
    )
)

# --------------------------
# 4. COMBINE BOTH
# --------------------------

correlation_all = (
    (scatter_all + smooth_all)
    .properties(
        width=900,
        height=550,
        title="Relationship between Earthquake Frequency and House Prices (All Years, Log Scale)"
    )
    .configure_axis(
        gridOpacity=0.20,
        labelFontSize=12,
        titleFontSize=14
    )
    .configure_title(
        fontSize=22,
        anchor="start"
    )
)

correlation_all

alt.LayerChart(...)

In [11]:
import altair as alt
import numpy as np
import pandas as pd

# --------------------------
# 1. CLEAN DATA
# --------------------------

df_corr = df_state_aggreg.copy()

# Remove invalid values
df_corr = df_corr[
    (df_corr["n_earthquakes"] > 0) &
    (df_corr["avg_price"] > 0) &
    (df_corr["avg_magnitude"].notna())
].copy()

# Add log value for regression
df_corr["log_eq"] = np.log(df_corr["n_earthquakes"])

# --------------------------
# 2. CREATE DYNAMIC EXCLUSIVE BINS
# --------------------------

# Determine min and max of avg_magnitude
min_mag = df_corr["avg_magnitude"].min()
max_mag = df_corr["avg_magnitude"].max()

# Create 5 equal-width bins
bins = np.linspace(min_mag, max_mag, num=6)  # 6 edges → 5 bins
labels = [f"{round(bins[i],1)}-{round(bins[i+1],1)}" for i in range(len(bins)-1)]

# right=False ensures upper boundary is exclusive
df_corr["magnitude_bin"] = pd.cut(df_corr["avg_magnitude"], bins=bins, labels=labels, right=False)

# --------------------------
# 3. SCATTER WITH DARK COLORS
# --------------------------

color_scale = alt.Scale(
    domain=labels,
    range=["#7f3c8d", "#11a579", "#3969ac", "#f2b701", "#e73f74"]  # dark, distinct
)

scatter_all = (
    alt.Chart(df_corr)
    .mark_circle(size=90, opacity=0.7)
    .encode(
        x=alt.X(
            "n_earthquakes:Q",
            title="Earthquakes (log scale)",
            scale=alt.Scale(type="log"),
            axis=alt.Axis(format="~s")
        ),
        y=alt.Y(
            "avg_price:Q",
            title="Median House Price ($)",
            scale=alt.Scale(zero=False, padding=10)
        ),
        color=alt.Color(
            "magnitude_bin:N",
            title="Avg Magnitude Bin",
            scale=color_scale
        ),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("n_earthquakes:Q", title="Earthquakes"),
            alt.Tooltip("avg_price:Q", title="House Price ($)"),
            alt.Tooltip("avg_magnitude:Q", title="Avg Magnitude"),
            alt.Tooltip("magnitude_bin:N", title="Magnitude Bin")
        ]
    )
)

# --------------------------
# 4. TRENDLINE
# --------------------------

smooth_all = (
    alt.Chart(df_corr)
    .transform_regression("log_eq", "avg_price")
    .mark_line(color="black", size=3)
    .encode(
        x=alt.X("n_earthquakes:Q", scale=alt.Scale(type="log")),
        y="avg_price:Q"
    )
)

# --------------------------
# 5. COMBINE BOTH
# --------------------------

correlation_all = (
    (scatter_all + smooth_all)
    .properties(
        width=900,
        height=550,
        title="Earthquake Frequency vs House Prices (Dynamic Magnitude Bins)"
    )
    .configure_axis(
        gridOpacity=0.20,
        labelFontSize=12,
        titleFontSize=14
    )
    .configure_title(
        fontSize=22,
        anchor="start"
    )
    .configure_legend(
        titleFontSize=14,
        labelFontSize=13,
        symbolSize=150
    )
)

correlation_all



alt.LayerChart(...)

In [12]:
# Base scatter
base = alt.Chart(df_state_aggreg).mark_circle(size=60, opacity=0.5).encode(
    x=alt.X("n_earthquake:Q", scale=alt.Scale(type="log"), title="Earthquakes (log scale)"),
    y=alt.Y("avg_price:Q", title="Median House avg_price ($)"),
    tooltip=["State:N", "Year:O", "n_earthquake:Q", "avg_price:Q"]
)

# Régression linéaire par année
smooth = alt.Chart(df_state_aggreg).transform_regression(
    "n_earthquake", "avg_price", groupby=["Year"]  # par année
).mark_line(size=2, color="black").encode(
    x=alt.X("n_earthquake:Q", scale=alt.Scale(type="log")),
    y="avg_price:Q"
)

# Scatter + ligne
layered = base + smooth

# Facet par année
chart = layered.facet(
    column=alt.Column("Year:O", title="Year")
).properties(
    title="House Prices vs Earthquakes – Linear Trend per Year"
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    gridOpacity=0.2
).configure_title(
    fontSize=18,
    anchor="start"
)

chart

ValueError: DataFusion error: Schema error: No field named "Year". Valid fields are count, _vf_order, _agg_0, _agg_1.
    Context[0]: Failed to query node values


alt.FacetChart(...)

# Focus sur un état (Californie ou autre)

In [15]:
import altair as alt
from vega_datasets import data

# TopoJSON
states = alt.topo_feature(data.us_10m.url, "states")
counties = alt.topo_feature(data.us_10m.url, "counties")

# Filtrer Alaska + California
df_state_AC = df_state_aggreg_year[df_state_aggreg_year["state"].isin(["Alaska", "California"])].copy()
df_county_AC = df_county_aggreg_year[df_county_aggreg_year["state"].isin(["Alaska", "California"])].copy()

# 1) --- STATE EARTHQUAKE MAP ---
map_state_eq = (
    alt.Chart(states)
    .mark_geoshape()
    .encode(
        color=alt.Color(
            "n_earthquakes:Q",
            scale=alt.Scale(range=["#ffe6e6", "#800000"]),
            title="Earthquakes"
        ),
        tooltip=["state:N", "n_earthquakes:Q"]
    )
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_state_AC, "fips", ["state", "n_earthquakes"])
    )
    .project("albersUsa")
    .properties(title="Earthquakes – Alaska & California", width=400, height=300)
)

# 2) --- STATE PRICE MAP ---
map_state_price = (
    alt.Chart(states)
    .mark_geoshape()
    .encode(
        color=alt.Color(
            "avg_price:Q",
            scale=alt.Scale(range=["#e6f2ff", "#0055aa"]),
            title="Avg House Price"
        ),
        tooltip=["state:N", "avg_price:Q"]
    )
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_state_AC, "fips", ["state", "avg_price"])
    )
    .project("albersUsa")
    .properties(title="Avg Price – Alaska & California", width=400, height=300)
)

# 3) --- COUNTY EARTHQUAKE MAP ---
map_county_eq = (
    alt.Chart(counties)
    .mark_geoshape(stroke="white", strokeWidth=0.25)
    .encode(
        color=alt.Color(
            "n_earthquakes:Q",
            scale=alt.Scale(range=["#ffe6e6", "#800000"]),
            title="Earthquakes"
        ),
        tooltip=["state:N", "n_earthquakes:Q"]
    )
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_county_AC, "county_fips", ["county_fips", "n_earthquakes"])
    )
    .project("albersUsa")
    .properties(title="County Earthquakes – Alaska & California", width=400, height=300)
)

# 4) --- COUNTY PRICE MAP ---
map_county_price = (
    alt.Chart(counties)
    .mark_geoshape(stroke="white", strokeWidth=0.25)
    .encode(
        color=alt.Color(
            "avg_price:Q",
            scale=alt.Scale(range=["#e6f2ff", "#0055aa"]),
            title="Avg House Price"
        ),
        tooltip=["state:N", "avg_price:Q"]
    )
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_county_AC, "county_fips", ["county_fips", "avg_price"])
    )
    .project("albersUsa")
    .properties(title="County Prices – Alaska & California", width=400, height=300)
)

# Combine – 2×2 grid
final_maps = (map_state_eq | map_state_price) & (map_county_eq | map_county_price)
final_maps

alt.VConcatChart(...)

In [14]:
df_county_aggreg_year.columns


Index(['state', 'county_fips', 'year', 'n_earthquakes', 'avg_magnitude',
       'max_magnitude', 'avg_depth', 'n_properties', 'avg_price',
       'median_price', 'avg_bedrooms', 'avg_bathrooms', 'fips'],
      dtype='object')